<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/SSA_%EC%8B%A4%ED%97%98%EC%8B%A4_%EA%B3%B5%EC%9C%A0%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 시작

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/파이널 프로젝트/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 추가 import

In [3]:
# 경고출력 무시
import warnings
# warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore')

# # 하이퍼파라미터 튜닝용
# !pip install optuna
# import optuna

# 트랜스포머
!pip install transformers

# gc
import gc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.6 MB/s eta 0:00:00


# 함수 정리

In [4]:
# 맞춤법
!git clone https://github.com/ssut/py-hanspell.git
!cd py-hanspell
!pip install py-hanspell/

from hanspell import spell_checker

# 맞춤법 검사 함수
def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

print('맞춤법 준비 완료.')
print('=' * 100)


# 유사도 함수
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(input_text, generated_text):
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    similarity_score = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0] # cosine
    # similarity_score = pairwise_distances([input_embedding], [generated_embedding], metric='euclidean')[0][0] # euclidean
    # similarity_score = pairwise_distances([input_embedding], [generated_embedding], metric='manhattan')[0][0] # manhattan

    return similarity_score

print('유사도 함수 준비 완료.')
print('=' * 100)


# Paraphrase
from transformers import pipeline
pipe = pipeline("text2text-generation", model="lcw99/t5-base-korean-paraphrase")

def paraphrase_text(text):
    generated_text = pipe(text, max_length=512, early_stopping=True,
                          num_beams=4,
                          num_return_sequences=2,
                          no_repeat_ngram_size=4,
                          top_k=50,
                          top_p=0.95)
    generated_text = generated_text[0]['generated_text']
    return generated_text

print('Paraphrase 준비 완료.')
print('=' * 100)


# 한글자씩 출력
import time

#sentence = 출력할 문자, timesl = 글자가 출력되는 시간간격(단위:초)
def hgj (sentence, timesl=0.5):
    for i in range(len(sentence)):
        print(sentence[i], end="")
        time.sleep(timesl)

print('"한글자씩 출력" 준비 완료.')
print('=' * 100)

print('< 사전 함수 준비 완료.>')

Cloning into 'py-hanspell'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 110 (delta 16), reused 9 (delta 9), pack-reused 88
Receiving objects: 100% (110/110), 22.30 KiB | 2.48 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Processing ./py-hanspell
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=7442d0c07b9ce3b2c6324bc6f61f866800b862f78523ae2eba6cc8ecf42913e6
  Stored in directory: /root/.cache/pip/wheels/a3/a7/16/8db596361f402638188412108152b8fbf6db76c898e00fe02d
Successfully built py-hanspell
맞춤법 준비 완료.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 763.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.w

유사도 함수 준비 완료.


Paraphrase 준비 완료.
"한글자씩 출력" 준비 완료.
< 사전 함수 준비 완료.>


# 모델 예시

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'skt/kogpt2-base-v2'

model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

# 평가

## 평가 데이터 불러오기

In [8]:
# ssa_df = pd.read_csv(f'{DATA_PATH}SSA_random_df_50.csv')
# ssa_df.head()

# ssa_df = pd.read_csv(f'{DATA_PATH}SSA_random_df_50.csv')
# ssa_df.head()

ssa_lst = ['저번에도 말씀드렸는데 제발 탕수육소스좀 더주세요 항상부족해요',
 '진짜 솔직히 저번에 짬뽕밥 시켰는게 숫가락을 안줘서 전화하니까 요청사항에 안적어서 안줬다고해서 내탓이라길래 어이가없었는데 오늘은 단무지 빼고 달라는 요청사항도 무시하고 요청사항은 왜 적으라한거죠? 쟁반짜장에서 머리카락까지 나오고 대단하내요 참',
 '정말 아쉽네요.튀김옷 얇게 부탁드렸는데 역시나 튀김옷 반 고기반',
 '맛,양은 괜찮은데 늦은시간에 시켜서그런지,화가나 계셨던건지.문을 쌔게 열심히 두드리시더군요.도착해서 연락해주시면 받으러간다고 써놨었음이웃에 민폐될까 달려가서 열었는데 틱 음식주시고 쿨하게가시네요.이곳은 이번이 처음이자 마지막이겠다는생각이 듭니다.',
 '매운 음식을 좋아하고 불닭 볶음면도 우습게 느껴질 정도로 잘 먹는 편입니다.그런데 불짬뽕 한입 입에 넣자마자 아, 이건 못 먹겠는데? 라는 생각이 들었습니다.캡사이신을 너무 과하게 넣은 건지 먹을 수 있는 수준의 매움이 아니라 느껴집니다.그래도 시킨 게 아까워서 음료수로 입을 달래고 꾸역꾸역 몇 젓갈 먹었다가 하루종일 속 쓰리고 아파서 죽는 줄 알았습니다.불짬뽕인 건 알지만 직접 먹어보시든 제발 적당히 캡사이신, 매운맛 조절 해주세요!',
 '사장님 다 좋은데요 지금 그릇을 한 사흘넘게 안 가져가고 계세요 악취풍기기 일보직전이에요',
 '간짜장시켰는데 양파가 덜 익었는지 너무 맵고 아삭했어요 탕수육은 맛있었습니다',
 '양도 많고 맛도 있었는데 배달원분이 비교적 불친절했어요.',
 '맛이 좀 싱겁네요 그리고 반반 곱빼기를 시켰는데도 양이 그냥 일반으로 잘 못온줄알았음',
 '진짜 솔직히 맛이 없다.먹으면서 화가났다.',
 '주문한 메뉴와 다른 음식이 왔고음식맛이 솔직히 맛있다는 생각은 안들어요.낮이랑 저녁에 퀄리티가 다른건지 모르겠네요',
 '짬뽕은 속풀이용으로 좋네요',
 '면이 오래 담겨 있었는지 흐물흐물.국물도 볶음밥에 딸려오는짬뽕국물이 더 맛있어요.면류는 어쩔수 없나.볶은밥는 맛있었어요',
 '종종 시켜먹는집인데요.짜장이 쫌 불었었어요맛은 있었어요',
 '삼겹김친데 김치가하나도없엇네요.김치빠트리셧나봐요시간상 그냥먹었어요',
 '배달 중국집의 정석 같습니다',
 '새벽에 간짜장이 먹고싶어서 주문했던거였는데짜장이 너무짜서 놀랐고매장으로 전화해서 알려드려야하나 고민많이 하다가 너무 이른시간이라 기분상하실까봐 못했. 짜장소스를 볶아놓은 사진보시면 확인소스를 쓰시더라구요.일반짜장과 간짜장의 차이는 바로볶는것의 차이인데, 이럴줄 알았다면 일반짜장을 시켰을것같아요  면과 소스 따로 담아주는 차이라면 더 비싼돈을 주고 간짜장을 시킬 이유가 없지요.간짜장 좋아하시는분들 꼭 참고하세요탕수육과 짬뽕매콤한편은 맛있었습니다.',
 '예전 달인각이라 몆번실망해도 이번은 아니겠지라고 기대하고 주문했는데.갈수록 맛이 없어지네요.짜장은 갈색에 니맛도 내맛도아니고 탕수육은 기름덩어리에 눅눅.배고프다던 아들.먹다 젓가락 놓았습니다.이젠 절대 주문안합니다.',
 '짬뽕맛이 좀 매워서 눈물콧물이 날정도였네요배달은 빨리와서 좋습니다',
 '다른분들은 친절하시던데 노란랜드로바 신으신분은 새벽에 배달오셔서 욕하더군요 배달원 교육좀시키십시요. 한그릇은 손님아닌가요 차라리 한그릇은 배달하지마시요. 금문도에 실망입니다',
 '세상맛없고 쪼를때로 쪼른 짬뽕국물 말라비틀어진 짬뽕속에 오징어 불어터진 짬뽕면 그러나 탕슉이랑 쟁반짜장 짱 맛잇어요',
 '새벽시간에 주문. 깐풍기. 이미해놓은거 전자렌지에 돌린듯함. 짬뽕. 너무 묽고 뭐 들어간게 없음. 김치국느낌.',
 '자주 시켜 먹는 곳인데 너무 맛있습니다. 양도 많은 편이고 가격도 착해서 직장에서 종종 시켜 먹었는데 역시나 오늘도 너무 맛잇게 잘 먹었습니다. 다만 배달대행업체직원으로 인해 기분이 너무 상하더군요. 바로고에서 오신듯한데 오자마자 짜증을 버럭 내더군요 음식 어디다 놔야해요! 하면서 짜증을 내시기에 당황스러웠지만 그러려니 넘겼네요 그러고 카드로 결제 하는 동안에 저희 근무지가 안에서 비밀번호를 눌러야 나갈수있는데 문 열어요 하며 또 명령조로 이야기 하더군요 글자로 전달이 잘 안 되었지만 굉장히 짜증난어투더군요 굉장히 불쾌했습니다',
 '30 35분걸린다고 나와서 나가기 전 시켰는데 52분걸려서 일도 늦게 출근 했구요! 삼선짬뽕,짜탕면곱 시켰는데 짬뽕,짜장은 그냥저냥 먹을 만.탕수육은 우선 고기 누린내에  뻑뻑하고 맛없.양은 적은 편 인듯 합니다 글구 짜탕면곱 시켰는데 어처구니가 없네요 진짜!곱배기 시켰는데 왜 양이 그릇의 반도 안되냐고 전화 드렸더니 주문 받고 점표 옮겨 쓰는 과정에서 실수가 있다며 1000원 돌려 드릴께요!우선 죄송합니다가 먼저 아닙니까?먹을려고 양에 맞춰 시켰는데 먹다 끈긴 느낌으로 출근했네요',
 '요청사항이나 메모확인 좀 해주세요 제발',
 '큰탕수육을 시키긴 했는데 양이진짜 푸짐합니다. 전에 세트메뉴에 좀 적은거같아서 따로 시켰는데. 굳뜨비가 오는날이라 습도가 높아 바삭함이 약간 덜하긴 하지만 고기살도 많고 만족해요짜장면도 맛나고, 짬뽕에 불맛이 저는 넘 좋네요잘먹었습니다',
 '배달도 빠르고 맛있게 잘 먹었습니다',
 '잘먹었급니다 맛있네여 귯굿',
 '깐풍기 생각보다 많이 주셔서 놀랬네요 맛있게 잘 먹었습니다',
 '사장님 잘 먹었습니다.',
 '잘 시켜먹고 갑니다',
 '너무너무너무 맛있었어요다음에 재주문 할게여',
 '새벽에 주문했는데 맛있었어요!',
 '배달도 친절하시고 맛도 좋아요근데. 탕수육 양이. 셋트이긴 하지만.  좀 적어요. 조금만 더 주세요',
 '말이 필요없음 자주 시켜서 =',
 '너무좋아요!맛나욥!',
 '배송도 적당하고 맛도 괜찮아요잘 먹었어요',
 '맛있게 잘 먹었습니다',
 '와우 배달20분짱빠르네욥',
 '양이적긴하나 맛있게먹엇어요',
 '점심에 시켰는데도 배달 빨라서 좋았어요 그런데 면이 사알짝 불어서 그래도 맛있게 먹었습니다',
 '중국집인데 돈까스는 왜이렇게 맛있어',
 '근처에서 시켜먹는거보다 맛있어요!',
 '맛있어요! 양도 많아요!',
 '곱배기 시켰는데 양도 너무 많고 맛도좋고 배달도 빠르고 배달 최소요금도 저렴하구, 정말  추천드립니다.',
 '너무 맛있어요 비결이 궁금하네요 오늘도 시킬거에요',
 '처음 시켜보았는데맛도 괜찮았던거같구요.배달은 빨리왔네요.암튼 잘 먹었습니다',
 '항상 감사합니다      ',
 '맛있게 잘 먹었습니다',
 '늘 맛있게 먹어요.재활용 가능한 일회용그릇에 주셔서 깔끔하고 좋아요']

## 평가 모델

In [11]:
model = model.from_pretrained(f'{DATA_PATH}models/S/sohyun_23762_99_loss:0.1528').to(device)

## 실험 모델

In [12]:
lst = ['ㄱ','ㄴ','ㄷ','ㄹ','ㅁ','ㅂ','ㅅ','ㅇ','ㅈ','ㅊ','ㅌ','ㅍ','ㅎ',
      'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

total_lst = [] # 임시리스트를 담아 데이터프레임을 만들 최종 리스트

@torch.no_grad()
def edit_chatbot_2(model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    cnt_num = 0
    tmp_lst = []

    for review in ssa_lst:
        cnt_num += 1
        text = review.strip()
        print(f'{cnt_num}번 리뷰')
        print('review >> ', text)
        tmp_lst.append(text) # 데이터프레임을 위한 리뷰 저장

        if len(text) == 0:
            print("리뷰를 입력해주세요.")
        elif text[0] in lst:
            print('소중한 리뷰 감사합니다.')
        else:
            # print('입력값 >> ', correct_spell(text))
            # print()
            pass

            text = '<q>' + text + '</s><a>'
            x = tokenizer.encode(text, return_tensors='pt').to(device)
            q_len = len(text) +1

            best_similarity_score = -1.0
            generated_texts = []
            for i in range(num_samples):
                result_ids = model.generate(x,
                                    max_length=max_len, # 생성 최대 길이
                                    repetition_penalty=1.8, # 반복 토큰 생성에 대한 패널티, 1보다 큰 수를 줘야함
                                    use_cache=True, # 빠른 추론을 위한 캐시 여부
                                    do_sample=True, # 확률적 샘플링
                                    temperature=0.8, # 소프트맥스 온도 (1미만: 확률들의 편차가 높아짐 / 1초과: 확률들 편차 줄어듬)
                                    top_k=20
                                    )
                generated_text = tokenizer.decode(result_ids[0])
                generated_text = generated_text[q_len:-4]
                generated_text = generated_text.strip() # 답변에서 앞뒤 공백이 생기는 경우를 발견함
                # generated_text = paraphrase_text(generated_text)

                similarity_score = evaluate_similarity(text, generated_text)
                generated_texts.append((similarity_score,generated_text))
                print()
                print(generated_texts[i])

                if similarity_score > best_similarity_score:
                    best_similarity_score = similarity_score
                    best_generated_text = generated_text

                # best_generated_text = correct_spell(best_generated_text)

            tmp_lst.append(best_generated_text) # 데이터프레임을 위한 베스트답변 저장

            print()
            print('-' * 100)
            print("< Best_reply >\n", best_generated_text)
            print('-' * 100)
            print()
            print("Best_similarity_score:", best_similarity_score)
            print('text length: ', len(best_generated_text))
            print()

            if len(best_generated_text) > 100:
                # print('답변이 길어 Paraphrase 필요')
                # paraphrased_best_generated_text = paraphrase_text(best_generated_text)
                # # paraphrased_best_generated_text = correct_spell(paraphrased_best_generated_text)
                # tmp_lst.append(paraphrased_best_generated_text) # 데이터프레임을 위한 paraphrase답변 저장
                # total_lst.append(tmp_lst) # paraphrase답변까지 추가 후 최종리스트에 추가
                # tmp_lst = []

                # # 옵션
                # a = '...'
                # hgj(a, 0.5)
                # print()

                # print('-' * 100)
                # print('< Paraphrase 변환 리뷰 >\n', paraphrased_best_generated_text)
                # print('-' * 100)
                # print()
                pass
            else:
                print('Paraphrase 필요 없음.')
                print()
                total_lst.append(tmp_lst) # paraphrase가 필요 없으므로 없는 채로 최종리스트에 추가
                tmp_lst = []

            if cnt_num != 50: # 검사할 개수 조절
                print("=" * 100)
                print()
            else:
                print()
                print('=' * 50, '< 답변 변환 완료 >', '=' * 50)
                # pass
                break



    # 모은 답변들 데이터프레임으로 만들기
    print()
    b = '      데이터프레임 생성중...'
    hgj(b, 0.2006464555557)
    print()
    sum_num = 0
    for tmp_sample in total_lst:
        sum_num += len(tmp_sample)

    if sum_num == len(total_lst)*2:
        columns = ['고객리뷰', '사장답변']
    else:
        columns = ['고객리뷰', '사장답변', '전환답변']

    ssa_test_df = pd.DataFrame(total_lst, columns=columns)

    print()
    print('=' * 50, '<  생 성 완 료  >', '=' * 50)
    print()
    return ssa_test_df

In [ ]:
edit_chatbot_2(model, tokenizer, 128, device, 5)